In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Dolly Parton,I Dreamed Of A Hillbilly Heaven,I dreamed I was there in hillbilly heaven oh w...
1,Falco,Falco Rides Again,"These are the given lyrics for this song, but ..."
2,Carpenters,This Masquerade,Are we really happy with \r\nThis lonely game...
3,Jimi Hendrix,3Rd Stone From The Sun,"\r\n'Star fleet to scout ship, please give y..."
4,Vince Gill,White Christmas,I'm dreaming of a white Christmas \r\nJust li...
5,Kenny Rogers,All God's Lovely Children,"A little baby's born, daddy's up and gone \r\..."
6,Gucci Mane,Hustle,[Chorus] \r\nSince you can't get right Imma s...
7,Dusty Springfield,Bad Case Of The Blues,Such a bad case of the blues \r\n \r\nI thou...
8,X,Little Honey,"Little honey, won't you come over here? \r\nL..."
9,Ray Boltz,Reigning,"Reigning \r\nWords by Ray Boltz, Music by Ray..."


In [9]:
df['text'][0]

"I dreamed I was there in hillbilly heaven oh what a beautiful sight  \r\nAnd I met all the stars in hillbilly heaven oh what a star spangled night  \r\nLast night I dreamed I went to hillbilly heaven  \r\nAnd just as I arrived the old curtain lifted  \r\nAnd there on stage stood who else but Roy Acuff and Tex Ritter  \r\nThey said well how you doing come on in here  \r\nI know there's some folks you want to see  \r\nOver there's our newest member your old friend Conway Twitty say Hello Darlin'  \r\nAnd there's gentlemen Jim Reeves singin' with heaven's choir  \r\nRoger Miller he just got here he's making everybody laugh  \r\nCourse you know Roy Orbison can sing just as high as anybody here  \r\nAnd Ernest Tubb can sing as low  \r\nAnd Patsy Cline well bet she's the best singin' angel here  \r\nAnd Dottie West is probably the prettiest  \r\nThere's the old pea-picker Ernie Ford  \r\nJimmy Rodgers George Morgan Hank Williams  \r\nAnd the wandering boy Web Pierce  \r\nThe old honky tonke

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.01058172, 0.01796249, ..., 0.03202339, 0.01949242,
       0.03832483])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text
2784,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Crying Over You')

['My Eyes Are Crying For You',
 'Cry Cry Darling',
 'Cry Baby',
 'Crybaby',
 "Isn't Life Strange",
 'Cry Me A River',
 "Cry 'til My Tears Run Dry",
 'Pretty When You Cry',
 'I Went To Your Wedding',
 'Seven Lonely Days',
 'Spread A Little Happiness',
 'Fool To Cry',
 'Bird Song',
 'Devil Cried',
 "Big Girls Don't Cry",
 'When The Time Comes',
 'Laughing On The Outside',
 "I'm So Lonesome I Could Cry",
 'For Crying Out Loud',
 'Let Me Fall']

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))